In [6]:
import scipy.io
data = scipy.io.loadmat('D:/本科研究生资料/硕士研究生/课程/机器学习与数据科学/作业/IRIS_4x150_3class.mat')

In [7]:
print(data.keys())

dict_keys(['__header__', '__version__', '__globals__', 'X', 'name', 'label'])


In [8]:
def parse(data):
    data_set = data['X']
    label = data['label'][0]
    data_num = len(data_set[0])
    return data_set,label,data_num

data_set,label,data_num = parse(data)

In [9]:
from random import sample
def split_train_test(data_num):
    l = range(data_num)
    train_set = sorted(sample(l,int(data_num/2)))
    test_set = sorted(set(l) - set(train_set))
    return train_set,test_set

train_set,test_set = split_train_test(data_num)

In [10]:
def Odistance(x,y):
    distance = 0
    dimensions = len(x)
    for dim in range(dimensions):
        distance += (x[dim] - y[dim])**2
    return distance**0.5

#print(Odistance([1,1,1,1],[2,2,2,2]))

In [81]:
from cvxopt import matrix, solvers

def judge(k_list):
    k_index_list = [x[0] for x in k_list]
    class_ = [0]*4
    for index in k_index_list:
        class_[label[index]] += 1
    return class_.index(max(class_))

def judge2(k_list):
    class_ = [0]*4
    d1 = k_list[0][1]
    dk = k_list[-1][1]
    for index_distance in k_list:
        index = index_distance[0]
        di = index_distance[1]
        if dk - d1 != 0:
            wi = (dk - di)/(dk - d1)
        else:
            wi = 1
        class_[label[index]] += wi
    return class_.index(max(class_))

def judge3(k_list,x):
    k_index_list = [x[0] for x in k_list]
    k = len(k_index_list)
    class_ = [0]*4
    P = []
    q = []
    G = []
    h = []
    A = []
    b = []
    for index1 in k_index_list:
        p_row = []
        for index2 in k_index_list:
            p_row.append(2*(data_set[0][index1]*data_set[0][index2]+data_set[1][index1]*data_set[1][index2]+data_set[2][index1]*data_set[2][index2]+data_set[3][index1]*data_set[3][index2]))
        P.append(p_row)
        
    for index in k_index_list:
        q.append(-2*x[0]*data_set[0][index]-2*x[1]*data_set[1][index]-2*x[2]*data_set[2][index]-2*x[3]*data_set[3][index])
        
    for i in range(k):
        G_row = [0.0]*k
        G_row[i] = -1.0
        G.append(G_row)
        
    h = [0.0]*k
    A = [1.0]*k 
    b = [1.0]
    
    P = matrix(P)
    q = matrix(q)
    G = matrix(G)
    h = matrix(h)
    A = matrix(A,(1,k))#原型为cvxopt.matrix(array,dims)，等价于A = matrix([[1.0],[1.0]]）
    b = matrix([1.0])
    
    W = solvers.qp(P,q,G,h,A,b)['x']
    for i in range(k):
        class_[label[k_index_list[i]]] += W[i]
    return class_.index(max(class_))

In [26]:
train_set,test_set = split_train_test(data_num)

In [201]:
train_set,test_set = split_train_test(data_num)
k = 50
real_class_list = []
predict_class_list = []
test_num = len(test_set)
hit_num = 0
for index1 in test_set:
    real_class = label[index1]
    real_class_list.append(real_class)
    x = [data_set[0][index1],data_set[1][index1],data_set[2][index1],data_set[3][index1]]
    distance_index_list = []
    for index2 in train_set:
        y = [data_set[0][index2],data_set[1][index2],data_set[2][index2],data_set[3][index2]]
        distance = Odistance(x,y)
        distance_index_list.append((index2,distance))
    distance_index_list.sort(key = lambda x : x[1]) #从小到大
    k_list = distance_index_list[0:k]
    
    predict_class = judge3(k_list,x)
    predict_class_list.append(predict_class)
    if predict_class == real_class:
        hit_num += 1
        
print("准确率为：",hit_num)
print(real_class_list)
print(predict_class_list)

     pcost       dcost       gap    pres   dres
 0: -4.0259e+01 -4.1267e+01  5e+01  7e+00  1e-02
 1: -4.0257e+01 -4.1188e+01  2e+00  1e-01  3e-04
 2: -4.0234e+01 -4.0402e+01  2e-01  3e-03  5e-06
 3: -4.0247e+01 -4.0291e+01  5e-02  6e-04  1e-06
 4: -4.0256e+01 -4.0270e+01  1e-02  7e-17  1e-16
 5: -4.0259e+01 -4.0263e+01  4e-03  2e-16  1e-16
 6: -4.0260e+01 -4.0260e+01  5e-04  1e-16  1e-16
 7: -4.0260e+01 -4.0260e+01  3e-05  2e-16  1e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -3.5009e+01 -3.6044e+01  6e+01  7e+00  1e-02
 1: -3.5002e+01 -3.5943e+01  3e+00  3e-01  5e-04
 2: -3.4960e+01 -3.5283e+01  4e-01  2e-02  4e-05
 3: -3.4986e+01 -3.5070e+01  9e-02  1e-03  2e-06
 4: -3.5003e+01 -3.5044e+01  4e-02  8e-17  1e-16
 5: -3.5006e+01 -3.5021e+01  1e-02  2e-16  6e-17
 6: -3.5009e+01 -3.5014e+01  5e-03  2e-16  7e-17
 7: -3.5010e+01 -3.5010e+01  6e-04  1e-16  7e-17
 8: -3.5010e+01 -3.5010e+01  2e-04  1e-16  8e-17
 9: -3.5010e+01 -3.5010e+01  2e-05  6e-17  8e-1

 9: -3.1323e+01 -3.1323e+01  7e-05  1e-07  2e-10
10: -3.1323e+01 -3.1323e+01  7e-07  1e-09  2e-12
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -3.7890e+01 -3.8900e+01  5e+01  7e+00  1e-02
 1: -3.7888e+01 -3.8824e+01  2e+00  2e-01  4e-04
 2: -3.7864e+01 -3.8102e+01  3e-01  3e-03  6e-06
 3: -3.7877e+01 -3.7938e+01  6e-02  7e-04  1e-06
 4: -3.7887e+01 -3.7913e+01  3e-02  5e-05  9e-08
 5: -3.7889e+01 -3.7896e+01  7e-03  1e-05  2e-08
 6: -3.7890e+01 -3.7891e+01  2e-03  1e-16  6e-17
 7: -3.7890e+01 -3.7890e+01  1e-04  6e-17  8e-17
 8: -3.7890e+01 -3.7890e+01  1e-06  7e-17  9e-17
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -7.3699e+01 -7.4759e+01  6e+01  7e+00  8e-03
 1: -7.3693e+01 -7.4679e+01  2e+00  1e-01  1e-04
 2: -7.3675e+01 -7.3950e+01  3e-01  5e-03  5e-06
 3: -7.3696e+01 -7.3747e+01  5e-02  6e-04  6e-07
 4: -7.3700e+01 -7.3709e+01  9e-03  6e-05  6e-08
 5: -7.3700e+01 -7.3701e+01  8e-04  3e-06  3e-09
 6: -7.3700e+01 -7.3700

     pcost       dcost       gap    pres   dres
 0: -5.7979e+01 -5.9005e+01  5e+01  7e+00  8e-03
 1: -5.7974e+01 -5.8938e+01  2e+00  2e-01  2e-04
 2: -5.7952e+01 -5.8281e+01  3e-01  4e-03  5e-06
 3: -5.7973e+01 -5.8034e+01  6e-02  3e-16  1e-16
 4: -5.7979e+01 -5.7993e+01  1e-02  3e-16  1e-16
 5: -5.7980e+01 -5.7981e+01  5e-04  9e-17  1e-16
 6: -5.7980e+01 -5.7980e+01  5e-06  2e-16  1e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -6.0564e+01 -6.1590e+01  5e+01  7e+00  8e-03
 1: -6.0547e+01 -6.1528e+01  2e+00  9e-02  1e-04
 2: -6.0539e+01 -6.0725e+01  2e-01  3e-03  3e-06
 3: -6.0555e+01 -6.0601e+01  5e-02  3e-04  3e-07
 4: -6.0567e+01 -6.0579e+01  1e-02  1e-06  1e-09
 5: -6.0570e+01 -6.0571e+01  2e-03  6e-09  7e-12
 6: -6.0570e+01 -6.0570e+01  2e-04  8e-17  2e-16
 7: -6.0570e+01 -6.0570e+01  8e-06  8e-17  1e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -4.2459e+01 -4.3496e+01  6e+01  7e+00  1e-02
 1: -4.2429e+01 -4.3380e

     pcost       dcost       gap    pres   dres
 0: -1.2180e+02 -1.2280e+02  7e+01  8e+00  7e-03
 1: -1.2176e+02 -1.2246e+02  6e+00  7e-01  5e-04
 2: -1.2165e+02 -1.2183e+02  2e+00  2e-01  1e-04
 3: -1.2164e+02 -1.2177e+02  1e-01  2e-16  1e-16
 4: -1.2168e+02 -1.2172e+02  4e-02  1e-16  9e-17
 5: -1.2170e+02 -1.2171e+02  1e-02  3e-16  1e-16
 6: -1.2171e+02 -1.2171e+02  3e-03  2e-17  1e-16
 7: -1.2171e+02 -1.2171e+02  8e-05  2e-17  1e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -8.4997e+01 -8.6007e+01  6e+01  8e+00  7e-03
 1: -8.4987e+01 -8.5915e+01  4e+00  5e-01  4e-04
 2: -8.4893e+01 -8.5425e+01  5e-01  9e-17  1e-16
 3: -8.4939e+01 -8.5068e+01  1e-01  3e-17  8e-17
 4: -8.4973e+01 -8.5039e+01  7e-02  1e-16  1e-16
 5: -8.4990e+01 -8.5006e+01  2e-02  1e-16  1e-16
 6: -8.4997e+01 -8.5000e+01  3e-03  2e-16  2e-16
 7: -8.4999e+01 -8.4999e+01  3e-04  1e-16  2e-16
 8: -8.4999e+01 -8.4999e+01  1e-05  2e-16  2e-16
Optimal solution found.
     pcost       dcost 

In [202]:
print("准确率为：",hit_num)
print(real_class_list)
print(predict_class_list)

准确率为： 72
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [208]:
list_ = [69,60,70,69,75,64,73,74,60,72]
print(sum(list_)/750)
import numpy as np 
var = np.var(list_)
print((var**0.5)/75)

0.9146666666666666
0.06958927120373273
